**Folium** is probably my favorite mapping library making dynamic [Leaflet]( https://leafletjs.com/) maps. Here are a few examples using data from the Icelandic Met Office.
- Use a basemap
- Add a layer via WMS
- Add a layer via WFS with a query
- Add a layer via ArcGIS REST Service

Check out [Folium](https://python-visualization.github.io/folium/latest/) site for more mapping goodness.

In [1]:
import folium
import requests
import json
import os

In [2]:
# let's start by creating an empty map using the IMO basemap

# basemap tile url and attribution
basemap_tiles = "https://geo.vedur.is/geoserver/www/imo_basemap_epsg3857/{z}/{x}/{y}.png"
basemap_attribution = "Icelandic Met Office | National Land Survey of Iceland | © OpenStreetMap contributors"

#create map and assign options
map = folium.Map(location=[65, -19], zoom_start=6, tiles=basemap_tiles, attr=basemap_attribution)

#initiate and display map. You can also save as html with map.save("index.html")
map

In [3]:
# let's add a WMS layers and zoom in our map a bit
# we will use on of several glacier outlines available
# to get a full list of avaialble layers goto https://geo.vedur.is/geoserver/wms?request=GetCapabilities&service=WMS

map = folium.Map(location=[66.15, -22.2], zoom_start=11, tiles=basemap_tiles, attr=basemap_attribution)

# define parameters for a WMS layer
wms_overlay = folium.raster_layers.WmsTileLayer(
               url="https://geo.vedur.is/geoserver/wms", # url to wms server
               name="Glacier outlines around 1890", # name of wms layer for layer controls
               fmt='image/png',  # image/png is the default for format
               layers="glaciology:glacieroutline_lia", # name of wms layer
               transparent=True, # allow transparancy
               #opacity=0.5, # set transparancy
               overlay=True, # overlay wms layer on top of basemap
               )

wms_overlay.add_to(map)

map

In [4]:
# now let's try WFS

# constructing a request
base_url = 'https://geo.vedur.is/geoserver/ows?'
params = {
    'service':'WFS',
    'version': '1.0.0',
    'request': 'GetFeature',
    'typeName': 'meteorology:IcelandicMetOffice_CAP_areas',
    #'cql_filter': 'region_ord LIKE \'2%\'',
    'cql_filter': 'region_ord = 5',
    'outputFormat': 'application/json'
}

# send request
r = requests.get(base_url, params)
# lets take a look at the geoJSON
r.json()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'IcelandicMetOffice_CAP_areas.9',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-23.39, 65.6],
       [-23.65, 65.59],
       [-24.01, 65.71],
       [-24.11, 65.81],
       [-23.86, 66.06],
       [-23.27, 66.3],
       [-23.1, 66.56],
       [-22.87, 66.52],
       [-22.58, 66.47],
       [-22, 65.99],
       [-22, 65.7],
       [-22.54, 65.69],
       [-23.01, 65.65],
       [-23.39, 65.6]]]]},
   'geometry_name': 'the_geom',
   'properties': {'name_en': 'Westfjords',
    'name_is': 'Vestfirðir',
    'region_ord': 5}}],
 'totalFeatures': 1,
 'numberMatched': 1,
 'numberReturned': 1,
 'timeStamp': '2023-09-15T23:12:55.785Z',
 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}}}

In [5]:
# save response as json
data = r.json()
with open(os.path.join('data/','forecast_areas.json'), 'w') as f:
    json.dump(data, f)
f.close()

In [6]:
# make a new map and add the geoJSON from the previous WFS request
map = folium.Map(location=[65, -19], zoom_start=6, tiles=basemap_tiles, attr=basemap_attribution)
folium.GeoJson(data, name="Forecast areas").add_to(map)
folium.LayerControl().add_to(map)
map

In [7]:
base_url = 'https://geo.vedur.is/geoserver/ows?'
params = {
    'service':'WFS',
    'version': '1.0.0',
    'request': 'GetFeature',
    'typeName': 'tos:v_stations_wgs84',
    #'maxFeatures': '20',
    'cql_filter': 'subtype LIKE \'SIL%\' AND date_end IS NULL',
    'outputFormat': 'application/json'
}
r = requests.get(base_url, params)
r.json()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'v_stations_wgs84.fid--6249f597_18a67f52bce_2521',
   'geometry': {'type': 'Point', 'coordinates': [-21.7511, 64.30177]},
   'geometry_name': 'geom_wgs84',
   'properties': {'gid': 16074,
    'id_entity': 16074,
    'name': 'Morastaðir',
    'station_identifier': 'mor',
    'marker': 'mor',
    'lat': 64.30177,
    'lon': -21.7511,
    'altitude': 130,
    'domain': 'geophysical',
    'subtype': 'SIL stöð',
    'operational_class': 'A',
    'owner': None,
    'date_start': '2017-11-22T14:00:00Z',
    'date_end': None,
    'service_bulletin': None}},
  {'type': 'Feature',
   'id': 'v_stations_wgs84.fid--6249f597_18a67f52bce_2522',
   'geometry': {'type': 'Point', 'coordinates': [-22.05954, 63.91995]},
   'geometry_name': 'geom_wgs84',
   'properties': {'gid': 18389,
    'id_entity': 18389,
    'name': 'Móhálsadalur',
    'station_identifier': 'moh',
    'marker': 'moh',
    'lat': 63.91995,
    'lon': -22.05954,
  

In [8]:
data = r.json() # add data to get nested list
with open(os.path.join('data/', 'SIL_stations.json'), 'w') as f:
    json.dump(data, f)
f.close()

In [9]:
map = folium.Map(location=[65, -19], zoom_start=6, tiles=basemap_tiles, attr=basemap_attribution)
folium.GeoJson(data,
               name="Seismic stations",
               popup=folium.GeoJsonPopup(fields=['name']),
               tooltip=folium.features.GeoJsonTooltip(fields=['marker'])
              ).add_to(map)

folium.LayerControl().add_to(map)

map

In [10]:
# lets try using Esri ArcGIS REST API, similar to WFS
# documentation https://developers.arcgis.com/rest/
# taka look at https://luk.vedur.is/arcgis/rest/services for more services related to example below

# build request
base_rest_url = 'https://luk.vedur.is/arcgis/rest/services/ofanflod/flodautlinur_isn93/MapServer/0/query?'
rest_params = {
    'where':'FLOD=55434', # avalanche id picked
    'outFields': '*', # give me all the atrributes
    'f': 'geojson' # output should be geoJSON
}
rest_r = requests.get(base_rest_url, rest_params)
rest_r.json()

# can also use https://luk.vedur.is/arcgis/rest/services/ofanflod/flodautlinur_isn93/MapServer/0/query?where=FLOD=55434&outFields=*&f=geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 8566,
   'geometry': {'type': 'MultiLineString',
    'coordinates': [[[-23.508402667483587, 66.04845442352722],
      [-23.508328046575468, 66.04849451907099],
      [-23.508276644056043, 66.04853394049002],
      [-23.508211978624118, 66.04859745372721]],
     [[-23.51082238371245, 66.0495648205026],
      [-23.511036014531296, 66.04955075756305],
      [-23.511310039950537, 66.04951671691504],
      [-23.511434395730827, 66.0494609437256],
      [-23.51146712136236, 66.04938176121233],
      [-23.511424836483542, 66.0493403226909],
      [-23.511417368007898, 66.04929786369867],
      [-23.51144886936773, 66.04924479700338],
      [-23.511506907873315, 66.04920991943644],
      [-23.511589803214612, 66.0491838042287],
      [-23.511693035014606, 66.04914050240913],
      [-23.51183604149588, 66.04909129810459],
      [-23.51191727769143, 66.04905575534313],
      [-23.511949606939975, 66.04900740222915],
      [

In [11]:
# save data/request
data = rest_r.json() # add data to get nested list
with open(os.path.join('data/', 'avalanche_55434.json'), 'w') as f:
    json.dump(data, f)
f.close()

In [12]:
# cunstruct a map with the outline of avalanche 55434 with tooltip and some fields in popup
map = folium.Map(location=[66.05, -23.5], zoom_start=14, tiles=basemap_tiles, attr=basemap_attribution)
folium.GeoJson(data,
               name='Avalanche 55434',
               popup=folium.GeoJsonPopup(fields=['FLOD','ATHS_DAGS','ATHS_STUTT']),
               tooltip=folium.features.GeoJsonTooltip(fields=['FLOD'])
              ).add_to(map)

folium.LayerControl().add_to(map)

map